In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict, defaultdict
import os
import pyfpgrowth;
from efficient_apriori import apriori

datapath = os.getcwd()
log_data_path = datapath + "/data" #all log data should kept in this folder
result_save_path = datapath + "/result" # all generated txt result save here
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

In [2]:
chrome_log = log_data_path + "/Log.txt.access_log.2015"
data = pd.read_csv(chrome_log , sep=" ",header=None,names=["IP","Junk1","Junk2","DateTime","Permission","FilePath","Status","BytesAccessed","Junk3","URL"])

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
type(data)

pandas.core.frame.DataFrame

In [4]:
data=data.drop(['Junk1','Junk2','Junk3'], axis=1)

In [5]:
data['DateTimePermission'] = data['DateTime'].str.cat(data['Permission'])

In [6]:
data=data.drop(['DateTime','Permission'], axis=1)

In [7]:
#Remove HTTP request info from the filepaths
data['FilePath']=data['FilePath'].str.replace('GET ','')
data['FilePath']=data['FilePath'].str.replace('POST ','')
data['FilePath']=data['FilePath'].str.replace('HEAD ','')
data['FilePath']=data['FilePath'].str.replace('OPTIONS ','')
data['FilePath']=data['FilePath'].str.replace('PROPFIND ','')

In [8]:
data['FilePath']=data['FilePath'].str.replace(' HTTP/1.1','')

In [9]:
data['DateTimePermission']=data['DateTimePermission'].str.replace('-0500','')

In [10]:
data['DateTimePermission']=data['DateTimePermission'].str.replace('-0400','')

In [11]:
data=data.rename(index=str,columns={"DateTimePermission":"DateTime"})

In [12]:
data=data.drop(['Status'],axis=1)

In [13]:
#data = data.sort_values(by=['IP'])

In [14]:
"""
Pass the DF before any preprocessing to generate reply log and uncluster baseline txt
"""
def createReplyLog_And_BaseLine(dataFrame):
    replay_Log_Name = os.path.join(result_save_path, "replay_log_user.txt")
    freplay = open(replay_Log_Name,"w+")  
    for file in dataFrame.FilePath:
        freplay.write(file+"\n")
    freplay.close()

    #Write the cluster.index_base :: all unique filePath as unclustered: As baseline to compare our results
    baseCluster_File_Name = os.path.join(result_save_path, "cluster.index_baseline_user.txt")    
    fbaseCluster = open(baseCluster_File_Name,"w+")
    fbaseCluster.write("Unclustered Files:\n")
    for file in data.FilePath.unique():
        if file=="":
            continue
        fbaseCluster.write(file+"\n")

    fbaseCluster.close()

In [15]:
#reply Log and baseline cluster need to be created only Once, If u need uncomment below and run
#createReplyLog_And_BaseLine(data)

In [16]:
#Ritu's changes starts from here .....
print ("Before this preprocessing data size: ", data.FilePath.size)
data_filtered = (data.set_index('FilePath').filter(like='/~', axis=0).reset_index())[data.columns]
print ("After . preprocessing data size: ",data_filtered.FilePath.size)

Before this preprocessing data size:  1000000
After . preprocessing data size:  721282


In [17]:
data_filtered['DateTime'] = data_filtered['DateTime'].str.strip('[')
data_filtered['DateTime'] = data_filtered['DateTime'].str.strip(']')

In [18]:
#Sort the dataframe according to date
data_list = data_filtered['DateTime'].str.split(':')
dateList = []
for items in data_list:
    dateList.append(items[0])

data_filtered['SortedDateTime'] =pd.to_datetime(dateList)
data_filtered = data_filtered.sort_values(by=['SortedDateTime'], ascending = True)

In [19]:
#Get unique datelist from the SortedDateTime
uniqueDateList = []
uniqueDateList = data_filtered.SortedDateTime.unique()
print(len(uniqueDateList))

507


In [20]:
#Splitting the dates
uniqueDateListPart1 = uniqueDateList[:495]
uniqueDateListPart2 = uniqueDateList[495:497]
uniqueDateListPart3 = uniqueDateList[497:498]
uniqueDateListPart4 = uniqueDateList[498:499]
uniqueDateListPart5 = uniqueDateList[499:500]
uniqueDateListPart6 = uniqueDateList[500:502]
uniqueDateListPart7 = uniqueDateList[502:504]
uniqueDateListPart8 = uniqueDateList[504:505]
uniqueDateListPart9 = uniqueDateList[505:506]
uniqueDateListPart10 = uniqueDateList[506:507]

In [21]:
#DataFrames constructed by splitting dates
dataFramePart1 = data_filtered.loc[data_filtered.SortedDateTime.isin(uniqueDateListPart1)]
dataFramePart2 = data_filtered.loc[data_filtered.SortedDateTime.isin(uniqueDateListPart2)]
dataFramePart3 = data_filtered.loc[data_filtered.SortedDateTime.isin(uniqueDateListPart3)]
dataFramePart4 = data_filtered.loc[data_filtered.SortedDateTime.isin(uniqueDateListPart4)]
dataFramePart5 = data_filtered.loc[data_filtered.SortedDateTime.isin(uniqueDateListPart5)]
dataFramePart6 = data_filtered.loc[data_filtered.SortedDateTime.isin(uniqueDateListPart6)]
dataFramePart7 = data_filtered.loc[data_filtered.SortedDateTime.isin(uniqueDateListPart7)]
dataFramePart8 = data_filtered.loc[data_filtered.SortedDateTime.isin(uniqueDateListPart8)]
dataFramePart9 = data_filtered.loc[data_filtered.SortedDateTime.isin(uniqueDateListPart9)]
dataFramePart10 = data_filtered.loc[data_filtered.SortedDateTime.isin(uniqueDateListPart10)]
print ("Data Frame Shapes:")
print (dataFramePart1.shape)
print (dataFramePart2.shape)
print (dataFramePart3.shape)
print (dataFramePart4.shape)
print (dataFramePart5.shape)
print (dataFramePart6.shape)#2nd largest (We will avoid this two DF)
print (dataFramePart7.shape)#largest
print (dataFramePart8.shape)
print (dataFramePart9.shape)
print (dataFramePart10.shape)

Data Frame Shapes:
(49799, 6)
(90816, 6)
(39067, 6)
(47969, 6)
(47957, 6)
(105218, 6)
(161629, 6)
(46775, 6)
(60449, 6)
(71603, 6)


In [22]:
#Testing reset of indexes
#dataFramePart2 = dataFramePart2.reset_index(drop=True)
#dataFramePart2.FilePath[0]

In [23]:
"""
It generate 2 txt file: 
    1. Cluster List in txt with your given name of file
       cluster.index fileName Convention: cluster.index_algorithm_supportThreshold_timeWindow
       
    2. All filePath list from dataFrame with yoru given name of file
    
Args:
        patterns_keyList: List of list (each inner list presents a cluster)
        DataFrame_filepath: Data Frame file path list
        Algo_Name: Algorithm to generate this frequent pattern        
        supportThreshold :: used in the algorithm
        TimeWindow: which has been used in our transaction preprocessing
Optional Arguments: 
        Feature :: features consider to create transaction, ex : time only, time+directory
"""
def createClusterFile(patterns_keyList, DataFrame_filepath, algoName, supportThreshold, TimeWindow, feature =""):

    #---------------Results file path-----------------#    
    #append the result directory
    #DataPath_Name = os.path.join(result_save_path, DataPath_Name+".txt")

    cluster_File_Name = "cluster.index_" + algoName + "_Support=" + str(supportThreshold) + "_timeWindow="+ str(TimeWindow) + "_Features="+feature
    #append the result directory
    cluster_File_Name = os.path.join(result_save_path, cluster_File_Name+".txt")              
    print ("Cluster File Name: ", cluster_File_Name)
    #--------------------------------------------------#
    fCluster = open(cluster_File_Name,"w+")    
    count = 1;
    hash_Dict = {}
    for items in patterns_keyList:
        inner_keyList = list(items)
        if(len(inner_keyList)==1): ## remove single file cluster
            continue
        fCluster.write("Cluster %d\n" % (count))
        count = count + 1 
        for i in inner_keyList:
            #print(i)
            fCluster.write("%s\n" %i)
            if i not in hash_Dict:
                hash_Dict[i] = True    

    #create the unclustered filepath
    fCluster.write("Unclustered Files:\n")
    #filePath = open(DataPath_Name, "w+")

    #------------Uncluster Files in cluster.index and all FilePath in DataPath txt-------------#
    for i in DataFrame_filepath:
      #  filePath.write("%s\n"%(i.strip(' '))) #all data frame files
        if i not in hash_Dict:
            fCluster.write("%s\n"%i) #uncluster files
            hash_Dict[i] = True #Removing duplicates in uncluster list

    fCluster.close()  
    #filePath.close()

In [24]:
#Change the dataframe to include the ones you want time_transactions for

#Only change DF here to test with different DF
dataFramevar = dataFramePart1.reset_index(drop=True)

In [25]:
#Set a time window to prepare transactions:: timed_window no of different timed accessed files will be on same transaction.

#Number_of_transactions
#All secone based transaction will be in this list
time_transactions = []
time_transactions.clear()

#temp-> to be pushed into time_transactions
#tempTransactions = []
tempTransactions = set()
tempTransactions.clear()
print (type(tempTransactions))

previous_date_time = dataFramevar.DateTime[0] # just to avoid error
print ("Total Data Size: ", dataFramevar.FilePath.size)  

#sample data 
#print (data_filtered.FilePath[0])

timed_window = 5;  # Time Window -- Can set the number of different time window we want in one transaction
count_window = 0; #ref counting

for i in range (0,dataFramevar.FilePath.size-1):
    try:

        #if time-stamp is different then push the temporary list into the actual transaction
        if(previous_date_time != dataFramevar.DateTime[i]):
            count_window = count_window + 1
            #print ("Debug Log:",count_window, " ", data_filtered.DateTime[i]) 
            if(count_window>=timed_window):
             #   print ("\n\n")
                count_window = 0
              #  print ("Debug Log:",count_window, " ", data_filtered.DateTime[i]) 
                tempTransactionsList = list(tempTransactions)
                tempTransactionsList = [x.strip(' ') for x in tempTransactionsList]
                time_transactions.append(tempTransactionsList)
                # Need to convert before pushed into transaction
                tempTransactions.clear()  
                
            
            
        tempTransactions.add(dataFramevar.FilePath[i])                           
        previous_date_time = dataFramevar.DateTime[i] #update previous timestamp
        
    except Exception as e:
        print("Exception: ", e)


print ("No of Time Based Transactions: \n", len(time_transactions))

#print (time_transactions)


<class 'set'>
Total Data Size:  49799
No of Time Based Transactions: 
 7326


In [26]:
#Try FP_Growth algorithm on this transaction set
#print (time_transactions)

#Use find_frequent_patterns to find patterns in baskets that occur over the support threshold:
minSupport = 100
patterns = pyfpgrowth.find_frequent_patterns(time_transactions, minSupport)  # need to generate data using this param
print ("pattern:")

itemsetList = []
for key,val in patterns.items():
    itemsetList.append(key)
    
print ("\n\n\n")
print ("Number of cluster: ",len(itemsetList))
feature = "ServerLog1time"
createClusterFile(itemsetList, dataFramevar.FilePath, "FPGrowth",minSupport,timed_window,feature)
#Use generate_association_rules to find patterns that are associated with another with a certain minimum probability:
#rules = pyfpgrowth.generate_association_rules(patterns, 0.5)  # generate data using this prob 

#print("\nAssociation Rules :")
#for key,val in rules.items():
#    print (key , "=>" , val)


#Ritu's changes ends here

pattern:




Number of cluster:  353
Cluster File Name:  /Users/Mehedi/BitBucket/AdvancedOS Project/result/cluster.index_FPGrowth_Support=100_timeWindow=5_Features=ServerLog1time.txt


In [27]:
#Apurva's changes starts here ...
data=data.dropna()

In [28]:
data=data.drop(data.index[0])

In [29]:
data['DateTime']=pd.to_datetime(data['DateTime'],format='[%d/%b/%Y:%H:%M:%S]')

ValueError: time data '3100[11/Jan/2015:11:36:02' does not match format '[%d/%b/%Y:%H:%M:%S]' (match)

In [ ]:
data['EpochTime']=pd.DataFrame((data['DateTime'] - dt.datetime(1970,1,1)).dt.total_seconds())

In [ ]:
data

In [ ]:
datagroupby=data.groupby([pd.cut(data['EpochTime'],
                    np.arange(data['EpochTime'].min(),
                             data['EpochTime'].max(),900)),'IP','UserAgent'])

In [ ]:
dataTempGroupByList=[]
count=0
countsessions=0
for x in datagroupby.groups:
    try:
        datagroupby.get_group(x)
        dataTempGroupByList.append(datagroupby.get_group(x))
        countsessions=countsessions+1
    except KeyError:
        count=count+1

In [ ]:
dataTempGroupByList[49]

In [ ]:
countsessions
#Apurva's changes ends here ...

In [ ]:
#data_list = data_filtered['FilePath'].str.split('/')

In [ ]:
#userList = []
#for items in data_list:
#    userList.append(items[1])

In [ ]:
#userIdDF = pd.DataFrame({'userId': userList})
#data_filtered = data_filtered.join(userIdDF)

In [ ]:
#To check no of files accessed by a particular user
#data_filtered.loc[data_filtered['userId'] == '~duan']

In [ ]:
#Remove favicon.ico files as these doesn't represent a web page
#data_filtered = data_filtered[data_filtered.FilePath != '/favicon.ico']

In [ ]:
#Tested with just 30000 data
#data_random_sample = data_filtered.sample(30000)

In [ ]:
#data_random_sample

In [ ]:
#dataGroupByUserId=data_random_sample.groupby(['userId'])

In [ ]:
#dataGroupByList=[]
#for x in dataGroupByUserId.groups:
#    dataGroupByList.append(dataGroupByUserId.get_group(x)['FilePath'].tolist())

In [ ]:
#dataGroupByList